# Econometría Aplicada II
## Tarea 1
Importar librerías

In [ ]:
%%capture

# Don't crash
from IPython import get_ipython

# Clonar repo si estamos en colab
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/ArturoSbr/EmtrAp2-hw01
    # !pip install scipy==1.7.3
    #!pip install linearmodels
    %cd EmtrAp2-hw01/cod

# Libs
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.api import OLS
from linearmodels import PanelOLS
from matplotlib import pyplot as plt

Importar datos

In [ ]:
d1 = pd.read_csv('../dat/baseline.csv')
d2 = pd.read_csv('../dat/endline.csv')
d3 = pd.read_csv('../dat/completa.csv')

### 1.1. Balance
Tabla de balance por grupo de acuerdo a `T_nap`

In [ ]:
# Seleccionar 10 variables basales
X = ['time_in_office','age_','female_','education_','sleep_report','no_of_children_','act_inbed',
     'an_12_number_of_awakenings','an_13_average_awakening_length','unemployed']

# Inicializar lista
d = []

# Medias de variables basales
for x in X:
    # Grupos
    b, a = d1.groupby('T_nap')[x].apply(np.array)
    # t-test
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    # Agregar a lista
    d.append([x] + list(test))

# A tabla
t = pd.DataFrame(data=d, columns=['var','t','p']).sort_values('var')
t['var'] = t['var'].str.replace('_',' ')
t.round(3)

In [ ]:
# To latex
# print(t.set_index('var').to_latex(float_format='%.3f', caption='Balance sobre 10 covariables'))

Todos los p-values de las pruebas de diferencia de medias son mayores a 0.1. Esto significa que para toda variable independiente $X_j$, no podemos rechazar la hipótesis nula $\mu_{X_j}^C = \mu_{X_j}^T$. Las pruebas individuales sugieren que no existe ninguna diferencia entra las medias del grupo de control y el grupo de tratamiento.

Para evaluar la significancia de manera conjunta, uso el siguiente modelo de probabilidad lineal:
$$T_i = \beta_0 + X_i^T \beta + U_i$$
donde $X_i^T$ contiene todos los controles evaluados en la prueba anterior.

La prueba de hipótesis para determinar si al menos una de las variables indepentientes está relacionada con la asignación a tratamiento es:
$$H_0: \beta_1 = 0$$
$$H_1: \text{Al menos un coeficiente es distinto de cero}$$

In [ ]:
# T_nap en función de controles
m = OLS(endog=d1['T_nap'], exog=d1[X].assign(const = 1)).fit()

# Tabla de resumen
print(m.summary())

Al igual que en las diferencias de medias, todos los coeficientes estimados del modelo de probabilidad lineal tienen p-values menores a 0.1. Por ende, ninguno de las variables independientes tiene significancia individual para determinar la asignación a tratamiento.

La prueba que nos interesa se ve reflejada en el estadístico $F$. La prueba de significancia conjunta tiene un p-value de 0.882. Es decir, no hay evidencia de que alguno de los coeficientes sea distinto de cero.

Como conclusión, parece que el tratamiento sí fue asignado de manera aleatoria con base en los 10 controles que elegí.

### 1.2. Efectos de tratamiento
Efectos de tratamiento sobre la productividad de los trabajadores.

#### a) Estimadores de Neyman

In [ ]:
# Función para estimador de Neyman
def neyman(frame, treatment_col, values_col):
    # Arreglos C y T
    m = frame[[treatment_col,values_col]].notna().all(axis=1)
    a, b = frame[m].groupby(treatment_col)[values_col].apply(np.array)
    # Diferencia de medias
    tau = np.mean(b) - np.mean(a)
    # Error estándar sobre-estimado
    bse = np.sqrt(np.var(a, ddof=1) / len(a) + np.var(b, ddof=1) / len(b))
    # t-stat
    t = tau / bse
    # p-value
    p = 2 * (1 - stats.norm().cdf(np.abs(t)))
    return [treatment_col, tau, bse, t, p]

# Diferencia de Neyman
prod_ney = neyman(d2, 'T_nap', 'productivity')
print(f'Prueba de Neyman sobre productividad:\n{prod_ney}')

#### b) Estimadores OLS sin controles

In [ ]:
m = OLS(endog=d2['productivity'], exog=d2.assign(const = 1)[['const','T_nap']], missing='drop')
m = m.fit(cov_type='HC0')

# Diferencia OLS sin X
prod_ols = ['T_nap', m.params['T_nap'], m.bse['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']]
print(f'Estimación OLS sobre productividad:\n{prod_ols}')

#### c) Estimadores con controles
De acuerdo al paper, $X_i$ contiene `age_` en cuartiles, `female_` y la variable que indica si $i$ fue asignado a trabajar o a tomarse un break en vez de tomar una siesta.

Como esta pregunta usa la base con promedios durante los 20 días de estudio, la variable que indica la actividad asignada cada día a los individuos del grupo de control no está disponible. Por ende, usaré el siguiente model:
$$Y_i = \beta_0 + \sum_{q=1}^4I\big[\beta_q \times Q(age_i)\big] + \beta_f fem_i$$
donde $Q(age_i)$ asigna un cuartil a $i$ con base en su edad y $fem_i = 1$ si $i$ es mujer.

In [ ]:
# Edad a cuartiles y luego a dummies
d2['age_q'] = pd.qcut(x=d2['age_'], q=4, labels=[f'q{i}' for i in range(1,5)])
d2 = pd.get_dummies(data=d2, prefix='age_', prefix_sep='', columns=['age_q'], )

# Tratamiento y controles
X = ['T_nap','const','age_q2','age_q3','age_q4','female_']

# Correr regresión
m = OLS(endog=d2['productivity'], exog=d2.assign(const = 1)[X], missing='drop')
m = m.fit(cov_type='HC0')

# Regresión con controles
prod_ctr = ['T_nap', m.params['T_nap'], m.bse['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']]
print(f'Estimación OLS+X sobre productividad:\n{prod_ctr}')

#### d) Resultados a tabla

In [ ]:
# Concatenar resultados
t = pd.DataFrame(data=[prod_ney, prod_ols, prod_ctr],
                 columns=['Y','tau','se','t','p'],
                 index=['Neyman','OLS simple','OLS controles'])

# Tabla
t = t.drop(columns=['Y'])
t.round(3)

In [ ]:
# # To latex
# print(t.to_latex(float_format='%.3f', caption='Estimaciones de ATE sobre productividad', label='t2_ates_prod'))

Los tres métodos indican que tomar una siesta tiene un efecto negativo sobre la productividad de los trabajadores. Sin embargo, todos los efectos estimados carecen de significancia. Es decir, el efecto promedio de tratamiento no es estadísticamente distinto de cero.

Mi modelo preferido es el OLS con controles porque redujo el p-value del coeficiente asociado al tratamiento.

#### e) Nuevas variables dependientes

In [ ]:
# Crear índice de habilidades cognitivas
cog = ['corsi_measure','hf_measure','pvt_measure']
d1['cog'] = d2[cog].apply(lambda x: (x - x.mean()) / x.std()).mean(axis=1)
d2['cog'] = d2[cog].apply(lambda x: (x - x.mean()) / x.std()).mean(axis=1)

# Nuevas variables dependientes
Y = ['nap_time_mins','sleep_report','happy','cog','typing_time_hr']

# Correr regresiones
d = []
for y in Y:
    m = OLS(endog=d2[y], exog=d2.assign(const = 1)[X], missing='drop')
    m = m.fit(cov_type='HC0')
    d.append([m.params['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']])

# Resultados a tabla
t = pd.DataFrame(data=d, columns=['tau','t','p'], index=Y)

# Visualizar
t.transpose().round(3)

In [ ]:
# print(t.transpose().to_latex(float_format='%.3f', caption='ATE sobre otras variables', label='t1.2_ates_other'))

De acuerdo al modelo OLS con controles, el tratamiento:
1. Aumenta el promedio de minutos dormidos durante la siesta de 0 a 11.7 minutos
1. Aumenta el promedio de número de horas de sueño en 0.05 horas por día (pero no tiene significancia estadística)
1. Aumenta el promedio de la calificación de felicidad reportada en 0.05 puntos (pero no tiene significancia estadística)
1. Aumentar el índice promedio de desempeño cognitivo en 0.03 desviaciones estándar (pero no tiene significancia estadística)
1. Reducir el promedio de horas trabajadas en 0.02 unidades diarias (pero no tiene significancia estadística)

### 1.3. Fischer's Exact Test
#### a) Probar si el tratamiento tiene efecto nulo

In [ ]:
# fet = stats.permutation_test(data=(d2.loc[d2['T_nap'].eq(1) & d2['productivity'].notna(), 'productivity'],
#                                    d2.loc[d2['T_nap'].eq(0) & d2['productivity'].notna(), 'productivity']),
#                              statistic=lambda x, y: np.mean(x) - np.mean(y),
#                              n_resamples=1000,
#                              random_state=42)

# print('p-value:', round(fet.pvalue, 3))

#### b) Conclusión
Los p-values del efecto promedio estimado de tratamiento bajo Neyman y de OLS sin controles son 0.338 y 0.336 respectivamente. Con la falsificación de Fischer, el p-value es 0.308. Estos p-values son muy parecidos entre sí, por lo que podemos concluir con un alto grado de certeza que el efecto de las siestas sobre la productividad no es estadísticamente significativo.

Cuando agregamos controles al modelo OLS, el p-value baja un poco (0.206), pero permance sin significancia estadística.

En resumen, los cuatro métodos indican que las siestas no tienen un efecto estadísticamente significativo sobre la productividad de las personas.

### 1.4. Estratificación

In [ ]:
# Crear casos con datos basales
d1[['e','s']] = d1[['earnings','sleep_report']].apply(lambda x: (x >= x.median()).astype(int), axis=0)

# Agregar casos a `d2`
d2 = d2.merge(d1[['pid','e','s']], on='pid')

#### a) Número de observaciones asignadas a tratamiento y control en cada estrato

In [ ]:
t = d2.groupby(['e','s','T_nap']).size().reset_index(name='n')
t = t.pivot(index=['e','s'], columns='T_nap', values='n')
t.columns = ['ngC','ngT']
t

In [ ]:
# print(t.to_latex(caption='Observaciones por grupo', label='t1.4_strats'))

La tabla sí tiene los números que hubiera esperado ex-ante, pues los individuos asignados a tratamiento en cada grupo son prácticamente del mismo tamaño que el número de individuos asignado a control.

#### b) Efectos por estrato y agregados usando Neyman
Efectos por estrato

In [ ]:
# Inicializar lista
d = []

# Efecto por esstrato a cada variable
for y in Y:
    for e, s in [(0,0),(0,1),(1,0),(1,1)]:
        # Máscara
        m = d2['e'].eq(e) & d2['s'].eq(s)
        # ATE Neyman
        d.append([e, s, m.sum(), m.sum() / len(d2), y] + neyman(d2[m], 'T_nap', y))

# Resultados a tabla
t = pd.DataFrame(data=d,
                 columns=['e','s','ng','wg','depvar','indvar','tau','se','t','p'])
t = t.set_index('depvar')


# Visualizar resultados
ates = [t[['e','s','tau','se','p']]]

# Guardar resultados en lista para comparar más tarde
ates[0].round(3)

Efectos agregados

In [ ]:
# t = t['tau'].multiply(t['wg']).groupby(t.index.get_level_values(0)).sum().to_frame(name='tau')
t

In [ ]:
# To latex
# print(t.to_latex(float_format='%.3f', caption='Efectos agregados', label='t1.4_ates'))

#### c) Efectos estratificados con OLS
$$Y_i = \beta_0 + \beta_1 T_i + \beta_2 s_i + \beta_3 s_i T_i + \beta_4 e_i + \beta_5 e_i T_i + \beta_6 s_i e_i + \beta_7 s_i e_i T_i + U_i$$

In [ ]:
# Inicializar lista
d = []

# Regresión para todas las depvar
for y in Y:
    # Modelo
    formula = f'{y} ~ T_nap + s + I(s*T_nap) + e + I(e*T_nap) + I(e*s) + I(e*s*T_nap)'
    m = OLS.from_formula(formula=formula, data=d2).fit(cov_type='HC0')
    # Results table
    res = pd.concat([m.params, m.bse, m.tvalues, m.pvalues], axis=1).assign(depvar = y)
    d.append(res)

# Todos los modelos en una tabla
t = pd.concat(d, axis=0).reset_index()
t.columns = ['beta','value','bse','t','p','depvar']

# Coeficientes relevantes
t = t[t['beta'].isin(['T_nap','I(e * T_nap)','I(s * T_nap)','I(e * s * T_nap)'])]

# Aesthetics
t = t.set_index('depvar')

# Agregar a `ates`
ates.append(t[['beta','value','bse','p']])

# Visualizar
ates[1].round(3)

Resultados a tabla

In [ ]:
# Unir resultados
t = pd.concat(ates, axis=1).reset_index()
t = t.set_index(['depvar','e','s'])

# Visualizar
t.round(3)

In [ ]:
# print(t.to_latex(float_format='%.3f', caption='Efecto de tratamiento por estrato', label='t1.4_ates_g'))

Los coeficientes estimados son idénticos. Por ejemplo, la diferencia de medias en `happy` para el estrato con ganancias debajo de la media y sueño arriba de la media es 0.043. Podemos obtener el mismo valor estimado si sumamos los coeficientes $T + s T = 0.043$.

La única diferencia es en las pruebas de hipótesis. Mientras que con el estimador de Neyman solo tenemos que probar la hipótesis nula $H_0: \hat{\tau}_g = 0$, en el estimador con OLS tenemos que probar si la suma de los coeficientes que reconstruyen $\hat{\tau}_g$ es igual a cero usando un vector de restricción $l$.

Los errores estándar de los estimadores por OLS serán distintos que por el método de Neyman, pero en general son parecidos entre sí. Esto lo podemos ver de manera intuitiva al ver los renglones que corresponden al grupo que gana y duerme por debajo de la mediana. Los $p$-values de ambos estimadores son parecidos.

### 1.5. Atrición
#### a) Reportar atrición

In [ ]:
t = d2.groupby('T_nap')['drop_indicator'].agg(['size','sum'])
t['pct_int'] = t['sum'].div(t['size']) * 100
t['pct_ext'] = t['sum'].div(t['size'].sum()) * 100
t.round(2)

In [ ]:
# print(t.to_latex(float_format='%.2f', caption='Atrición', label='t1.5_attritors'))

#### b) Nuevo balance
Validez interna

In [ ]:
# Variables dependientes
X = ['time_in_office','age_','female_','education_','sleep_report','no_of_children_','act_inbed',
     'an_12_number_of_awakenings','an_13_average_awakening_length','unemployed']

# Tabla de balance
d = []
for x in X:
    b, a = d1[d1['drop_indicator'].eq(0)].groupby('T_nap')[x].apply(np.array)
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    d.append([x] + list(test))

# A tabla
t = pd.DataFrame(data=d, columns=['variable','t','p']).set_index('variable')
t.round(3)

In [ ]:
# print(t.to_latex(float_format='%.3f', caption='Validez interna', label='t1.5_internal'))

Prueba conjunta

In [ ]:
m = OLS(endog=d2.loc[d2['drop_indicator'].eq(0), 'T_nap'],
        exog=d2.loc[d2['drop_indicator'].eq(0), X].assign(const = 1),
        missing='drop').fit(cov_type='HC0')
print(f'p-value de significancia conjunta: {round(m.f_pvalue.item(), 3)}')

Sí tenemos un problema de validez interna porque después de la atrición los niveles promedio de educación y número de hijos son estadísticamente distintos entre los grupos de tratamiento y control. Si hacemos un $F$-test de significancia conjunta, rechazamos la hipótesis de que todos los coeficientes sean simultáneamente iguales a cero porque el $p$-value de la prueba es 0.095.

Validez externa

In [ ]:
# Probar si diferencia es significativa
d = []
for x in X:
    b, a = d1.groupby('drop_indicator')[x].apply(np.array)
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    d.append([x] + list(test))

# Resultados a tabla
t = pd.DataFrame(data=d, columns=['variable','t','p']).set_index('variable')
t.round(3)

In [ ]:
# print(t.to_latex(float_format='%.3f', caption='Validez externa', label='t1.5_external'))

Prueba de significancia conjunta

In [ ]:
m = OLS(endog=d1['drop_indicator'], exog=d1[X].assign(const = 1)).fit(cov_type='HC0')
round(m.f_pvalue.item(0), 3)

La atrición parece no haber afectado la validez externa de la muestra. Usando las pruebas individuales, la atrición tiene un $p$-value bajo para el número de hijos y mujeres. Sin embargo, ambas variables carecen de significancia estadística al 10%. Asimismo, la prueba conjunta resulta en un $p$-value muy cercano a cero, por lo que no podemos rechazar que todos los coeficientes sean distintos de cero simultáneamente. Es decir, parece que no existe un problema de validez externa.

#### c) Conclusión
La atrición fue sistemática entre el grupo de tratamiento y de control. Es decir, parece que el nivel de educación y el número de hijos determinan si alguien abandona o no el experimento. Esto nos lleva a un problema de validez interna porque los grupos de tratamiento y control después de la atrición no están balanceados.

Sin embargo, parece que la atrición no afectó la validez externa de la muestra, pues parece que las personas que abandonaron el estudio no afectaron las distribuciones de las variables de control. Ninguna de las 10 variables muestra una diferencia significativa antes y después de la atrición.

### 5. Lee Bounds
#### a) Perfiles
- Always Respondents: $S_i$ = 1 sin importar $T_i$
- Never Respondents: $S_i$ = 0 sin importar $T_i$
- Selective Respondents: $T_i = 0 \implies S_i = 0$, $T_i = 1 \implies S_i = 1$
- Counter-Selective Respondents: $T_i = 0 \implies S_i = 1$, $T_i = 1 \implies S_i = 0$

El supuesto de monotonicidad es que no existe alguno de los dos grupos de respuesta selectiva. En el contexto de este experimento, tiene sentido asumir que los Counter-Selective Respondents no existen porque el tratamiento es algo *bueno*. Es decir, si a alguien le toca tomar una siesta durante sus horas de trabajo, es razonable pensar que el tratamiento es algo deseable y por ende no incentivaría a los individuos a abandonar el experimento.

In [ ]:
# Columna S_i
d2['S'] = 1 - d2['drop_indicator']

# Casos
t = d2.groupby(['T_nap','S']).size()
t.unstack().transpose()

Calcular probabilidades

In [ ]:
# P(AR|T=0)
par = 128 / (81 + 128)

# P(SR|T=1)
psr = 182 / (182 + 23) - par

# Probabilidades
print(f'P(AR) = {par}', f'P(SR) = {psr}', f'P(NR) = {1 - (par + psr)}', sep='\n')



#### b) Lee Bounds

In [ ]:
# P(AR|S=1)
# par_s = par / d2['S'].mean()
par_s = par / (par + psr)

# Arreglos trat-cont dado S=1
a, b = d2[d2['S'].eq(1)].groupby('T_nap')['productivity'].apply(np.array)

# Lower bound
lb = b[b <= np.quantile(b, par_s)].mean() - a.mean()

# Upper bound
ub = b[b >= np.quantile(b, 1 - par_s)].mean() - a.mean()

# Bounds
print(f'El ATE de los AR está en [{lb, ub}]')

#### c) Comparación
Los resultados de la pregunta 2 no tienen por qué estar centrados en los Lee Bounds porque estiman el efecto para toda la población, mientras que el intervalo de esta pregunta acota el efecto de tratamiento para los Always Respondents.

Lo único que sí podemos ver es que el ATE observado en la muestra completa es mayor al ATE de los Always Respondents.

## 2. Matching
Favor de consultar arvhivo `matching.R`

## 3. Difference in Differences
### 3.1. Tabla 2x2
Uso baseline como `t0` y endline como `t1` ignorando atrición.

In [ ]:
# Pegar d1 & d2
t = pd.concat([d1[['pid','T_nap','productivity']].assign(t = 0),
               d2[['pid','T_nap','productivity']].assign(t = 1)],
              axis=0)

# Agrupar
t = t.groupby(['T_nap','t'])['productivity'].mean().reset_index(name='mean')

# Formato 2x2
t = t.pivot(index='t', columns='T_nap')

# Visualizar
t.round(2)

# To latex
# print(t.to_latex(float_format='%.2f', caption='Matriz de comparación', label='t3_2x2'))

Con los periodos $t \in \{0, 1\}$, el resultado anterior se puede estimando el coeficiente $\beta_3$ del modelo
$$prod_{it} = \beta_0 + \beta_1 T_i + \beta_2 t + \beta_3 t \times T_i + U_i$$

De acuerdo a la tabla, el ATE es:

In [ ]:
(t.iloc[1, 1] - t.iloc[0, 1]) - (t.iloc[1, 0] - t.iloc[0, 0])

### 3.2. 2x2
De acuerdo al paper, todos los individuos permanencieron en control durante los primeros ocho días del estudio y el experimento comenzó en el noveno día. Por ende, consideraré que el periodo $t = 0$ son las medias agrupadas de las observaciones de los primeros ocho días, y que el periodo $t = 1$ son las medias agrupadas de las observaciones de los días restantes.

In [ ]:
# Asignar tiempo binario
d3['t'] = np.where(d3['day_in_study'] <= 8, 0, 1)

# Agrupar por i, t para tener dos observationes por individuo
t = d3.groupby(['pid','t'])[['productivity','T_nap','age_','female_']].mean().reset_index()

# Verificar si todos tienen dos periodos observados
t.groupby('pid').size().agg(['min','max'])

Verificar que nadie tenga NANs

In [ ]:
t.isna().sum()

Quitar NaNs

In [ ]:
# Quitar a pid que tiene NAN en productivity
t = t[t['pid'] != t.loc[t['productivity'].isna(), 'pid'].item()]

# Conteo
print(f"Cada uno de los {t['pid'].nunique()} individuos tiene dos periodos ({len(t)} observaciones en total))")

Usando el método recién mencionado, conservo a más individuos que con las bases baseline y endline. Con esta nueva base, la tabla $2 \times 2$ queda así:

In [ ]:
# Agrupar
t2 = t.groupby(['T_nap','t'])['productivity'].mean().reset_index(name='mean')

# Formato 2x2
t2 = t2.pivot(index='t', columns='T_nap')

# Visualizar
t2.round(2)

# To latex
# print(t2.to_latex(float_format='%.2f', caption='Matriz de comparación (Versión 2)', label='t3_2x2_v2'))

Debido a que la tabla es un poco distinta, el TOT es un poco distinto por la naturaleza de la población. De acuerdo a esta tabla, el TOT es:

In [ ]:
(t2.iloc[1, 1] - t2.iloc[0, 1]) - (t2.iloc[1, 0] - t2.iloc[0, 0])

Regresión sin controles con el modelo
$$prod_{it} = \beta_0 + \beta_1 T_i + \beta_2 t + \beta_3 t \times T_i + U_i$$

In [ ]:
# Regresión con controles
m = OLS.from_formula('productivity ~ T_nap + t + I(t*T_nap)', t).fit()
m.params

Regresión con controles con el modelo
$$prod_{it} = \beta_0 + \beta_1 T_i + \beta_2 t + \beta_3 t \times T_i + age_i + female_i + U_i$$

In [ ]:
# Regresión con controles
m = OLS.from_formula('productivity ~ T_nap + t + I(t*T_nap) + age_ + female_', t).fit()
m.params

Para ambas regresiones debería usar errores clusterizados porque las variables observadas para un mismo individuo no son independientes ni idénticamente distribuidas entre ambos periodos de tiempo. La segunda instancia de cualquier variable es argumentablemente dependiente del su valor en el periodo de tiempo anterior.

### 3.3. Comparación
Los valores estimados del ATT en la productividad usando Matching varían entre $-953.26$ y $-814.67$. Los ATTs
usando cualquier estimador de Diferencia en Diferencias (DiD por sus siglas en inglés) resulta en valores positivos (entre $79.75$ y $87.56$).

En lo personal, confío más en el estimador de DiD, pues Matching funciona cuando puedes emparejar a la población con base en todas las covariables que podrían sesgar la diferencia de medias. En el caso de este estudio, hay muchos ejemplos de covariables no observables que podrían sesgar el efecto de las siestas sobre la productividad de aquellos individuos que recibieron el tratamiento. Por ejemplo, habilidad natural, conocimiento de computadoras, interés en el trabajo, etc. Como estas variables no son observables, no podemos medir si existe un desbalance en estos controles al emparejar por las variables que sí son observables. En caso de que Matching produzca desbalances en estas variables, el ATT estimado estará sesgado a pesar de estar balanceado en las variables por las cuales emparejamos a la población.

Al tomar la diferencia entre los periodos primero y segundo dentro de cada grupo (tratamiento o control), el estimador DiD es capaz de eliminar el sesgo por cualquier variable no observada siempre y cuando permanezca constante en el tiempo. Como el estudio duró 28 días, cualquier variación en el tiempo de las variables no observables sería negligible porque seguramente no cambiarían mucho dentro de un periodo de tiempo tan corto. Por ende, considero que los ATTs por el método de DiD es más confiable que el ATT usando Matching.

### 3.4. Figuras
#### 3.4.1. Series de tiempo

In [ ]:
# Crear grupos
d3['group'] = np.where(
    d3['T_nap'].eq(0),
    'No naps',
    np.where(
        d3['T_nap'].eq(1) & d3['treatment_group'].eq(0),
        'Nap only',
        np.where(
            d3['T_nap'].eq(1) & d3['treatment_group'].eq(1),
            'Nap and encouragement',
            'Nap and incentives'
        )
    )
)

# Minutos por grupo
t = d3.groupby(['group','day_in_study'])[['nap_time_mins']].mean().fillna(0).reset_index()

# Plot
for g in t['group'].unique():
    # Mask
    m = t['group'].eq(g)

    # Line plot
    plt.plot(t.loc[m, 'day_in_study'], t.loc[m, 'nap_time_mins'], label=g)

plt.axvline(8.5, color='black', ls='--')

# Aesthetics
plt.legend()
plt.xlim(2, 27)
plt.ylim(0, 17)
plt.xlabel('Day in Study')
plt.ylabel('Nap Duration (Minutes)')
plt.title('(d) Nap Sleep')

# Save
if 'google.colab' not in str(get_ipython()):
    plt.savefig('../fig/f341-nap-mins.png', dpi=200, bbox_inches='tight')

# Show
plt.show()

### 3.4.2 Promedios por rango de días
La gráfica que se intentó replicar necesitaba la variable Break Time para comparar a los individuos de tratamiento de aquellos que fueron asignados a tomar un descanso durante el timepo de siesta. Como esta variable no está disponible en ninguna de las bases, grafiqué los efectos de tratamiento tomando como grupo de referencia a los que no tomaron una siesta (trabajadores que se tomaron un descanso o que trabajaron durante las siestas).

In [ ]:
# Declarar grupos de días
d3['groupDay'] = np.where(
    d3['day_in_study'].le(8) | d3['day_in_study'].eq(28),
    'Bye',
    np.where(
        d3['day_in_study'].between(9, 12),
        '09-12',
        np.where(
            d3['day_in_study'].between(13, 15),
            '13-15',
            np.where(
                d3['day_in_study'].between(16, 18),
                '16-18',
                np.where(
                    d3['day_in_study'].between(19, 21),
                    '19-21',
                    np.where(
                        d3['day_in_study'].between(22, 24),
                        '22-24',
                        '25-27'
                    )
                )
            )
        )
    )
)

# Declarar variables promedio en baseline


# Convertir typing_time_hr a mins
d3['typing_time_mins'] = d3['typing_time_hr'] * 60

# Agrupar
m = d3['day_in_study'].between(9, 27)
t = d3[m].groupby(['groupDay','T_nap'])[['earnings','typing_time_mins','productivity']].agg(['mean','std','size'])

# Plots
fig, axes = plt.subplots(nrows=3, ncols=1)#, sharex=True)
fig.set_figheight(10)
fig.subplots_adjust(hspace=.5)

# Titles by case
ttls = {'earnings':'Earnings', 'typing_time_mins':'Minutes typing', 'productivity':'Productivity'}

for i, y in enumerate(['earnings','typing_time_mins','productivity']):
    # Get axis
    ax = axes.flatten()[i]

    # To frame
    d = t[y].reset_index()

    # Get tau
    d['tau'] = d['mean'] - d.groupby('groupDay')['mean'].shift()

    # Get std
    d['s'] = d['std'].div(d['size'])
    d['s'] = np.sqrt(d['s'] + d.groupby('groupDay')['s'].shift())

    # Keep second row per group
    d = d.loc[d['T_nap'].eq(1), ['groupDay','tau','s']]

    # Plot
    ax.errorbar(
        x=d['groupDay'], y=d['tau'], yerr=d['s'] * 1.96,
        ls='none', marker='^', elinewidth=1, capsize=4
    )

    # Plot aesthetics
    ax.axhline(0, color='black', lw=1)
    ax.set_xlabel('Day in Study')
    ax.set_ylabel(ttls[y])
    ax.set_title(ttls[y])
    
    # Save figure
    if 'google.colab' not in str(get_ipython()):
        plt.savefig('../fig/f341-Ys.png', dpi=200, bbox_inches='tight')

plt.show()

## 3.5. Tendencias paralelas

Tras eliminar las observaciones que abandonaron el experimento, uso evalúo las tendencias durante los ocho días previos al inicio del tratamiento usando el modelo:
$$Y_{it} = \beta_0 + \beta_1 T_i + \sum_{s < 0} \Big[ \delta_s DS_t + \tau_s DS_t \Big] + U_{it}$$

Mis hipótesis son:

$H_0: \tau_t = 0 \text{ } \forall \text{ } t \in \{-7, -6, -1\}$

$H_1:$ Cualquier otro caso

In [ ]:
# Recentrar en t = 8
d3['t'] = d3['day_in_study'] - 8

# Copiar d3
cols = ['pid','t','T_nap','productivity']
d = d3[cols].copy()

# Conservar a pid's SIN drop_indicator
d = d[d['pid'].isin(d2.loc[d2['drop_indicator'].eq(0), 'pid'].values)].reset_index(drop=True)

# Datos previos al tratamiento
t = d[d['t'] <= 0]
t = t.groupby(['T_nap','t'])['productivity'].mean().reset_index(name='y')

# Plot
for T in [0, 1]:
    label = {0:'Control',1:'Tratamiento'}[T]
    plt.plot(t.loc[t['T_nap'].eq(T), 't'], t.loc[t['T_nap'].eq(T), 'y'], label=label)

# Aesthetics
plt.title('Tendencias antes del tratamiento')
plt.xlabel('Días desde el tratamiento')
plt.ylabel('Productividad')
plt.legend()
try:
    plt.savefig('../fig/f35-parallel.png', dpi=200, bbox_inches='tight')
except:
    True
plt.show()

Preparar datos para PanelOLS

In [ ]:
d.head()

In [ ]:
# Crear copia de datos
t = d[d['t'] <= 0].copy()

# Crear dummies por periodo
s = pd.get_dummies(t[['t']], columns=['t'], drop_first=False, prefix_sep='')

# Declarar a t0 como el periodo de referencia
s.drop(columns=['t0'], inplace=True)

# Interacciones tiempoXtratamiento
s2 = s.multiply(t['T_nap'], axis=0)
s2.columns = [f'{col}*T_nap' for col in s2.columns]

# Pegar a datos
t = pd.concat([t, s, s2], axis=1)

# Asignar constante
t['const'] = 1

# PanelOLS Index
t.set_index(['pid','t'], inplace=True)

Ajustar modelo

In [ ]:
# Ajustar modelo
m = PanelOLS(t['productivity'], t.loc[:, 't-7':'const'])
m = m.fit(cov_type='clustered', cluster_entity=True)

# Resumen
print(m)

Comparar contra modelo restringido (modelo que asume que las pendientes son idénticas).

In [ ]:
# Restricted model
mR = PanelOLS(t['productivity'], t.loc[:, 't-7':'t-1'].assign(const = 1))
mR = mR.fit(cov_type='clustered', cluster_entity=True)

# F-test
Fstat = ((mR.resid_ss - m.resid_ss) / (m.df_model - mR.df_model)) / (mR.resid_ss / (mR.nobs - mR.df_model))
p = 1 - stats.f(dfn=7, dfd=m.df_resid).cdf(Fstat)
print(f'F-stat = {Fstat}')
print(f'p-value = {round(p, 3)}')

## 3.6 Matching y DiD
Rellené la serie para cada `pid` usando:
1. Las medias de los datos basales para missings en los primeros 8 días
2. La media móvil de los últimos 5 días para missings en los días 9 y adelante

Con series llena para todo `pid`, podemos hacer un match entre tratamiento y control usando controles vistos en Baseline. Luego, podemos usar los pesos para hacer el DiD.

In [ ]:
# Keep copy of non-dropped pids
s = d2.loc[d2['drop_indicator'].eq(0), 'pid'].values
d = d3[d3['pid'].isin(s)].copy()

# Define cog from baseline
m = d1[cog].mean()
s = d1[cog].std()
for x in cog:
    d[x] = (d[x] - m[x]) / s[x]
d['cog'] = d[cog].mean(axis=1)

# Columns to store in lean data
cols = [
    'pid','day_in_study','T_nap',                                                   # i-level
    'productivity','nap_time_mins','sleep_report','happy','cog','typing_time_hr'    # Targets
]

# Lean data
d = d[cols]
d = d.sort_values(['pid','day_in_study'])

# Columns to fill in by pid
cols = ['pid','productivity','nap_time_mins','sleep_report','happy','cog','typing_time_hr']

# Completar datos basales con medias basales
s = d1[cols].fillna(0)
s.columns = [col + '_B' for col in s.columns]
d = d.merge(s, left_on='pid', right_on='pid_B', how='inner')
for y in cols[1:]:
    d[y] = np.where(d['day_in_study'].le(8) & d[y].isna(), d[y + '_B'], d[y])

# Completar datos experimentales con moving averages
s = d.rolling(window=7, min_periods=0, on='pid', center=False)[cols[1:]].mean()
s.columns = [col + '_ma' for col in s.columns]
d = d.merge(s, left_on='pid', right_on='pid_ma', how='inner')
for y in cols[1:]:
    d[y] = np.where(d['day_in_study'].ge(9) & d[y].isna(), d[y + '_ma'], d[y])

# Eliminar columnas para rellenar
d = d.drop(columns=[col+'_B' for col in cols] + [col+'_ma' for col in cols])

# People who are still missing data
s = d[cols].isna().groupby(d['pid']).sum()

# Begone foul beasts
s = s[s.sum(axis=1) > 0].index.values
d = d[~d['pid'].isin(s)].reset_index(drop=True)

# Exportar para match en R
d.to_csv('../dat/unmatched.csv', index=False)

Traer relación de matchit

In [ ]:
# Matches and weights
s = pd.read_csv('../dat/matched.csv')

# Merge
d = d.merge(s, on='pid', how='inner')

Ver tendencias

In [ ]:
# Crear copia
s = d.copy()

# Recentrar en t = 8
s['t'] = s['day_in_study'] - 8

# Copiar d3
s = s[['pid','t','T_nap','productivity']]

# Datos previos al tratamiento
t = s[s['t'] <= 0]
t = t.groupby(['T_nap','t'])['productivity'].mean().reset_index(name='y')

# Plot
for T in [0, 1]:
    label = {0:'Control',1:'Tratamiento'}[T]
    plt.plot(t.loc[t['T_nap'].eq(T), 't'], t.loc[t['T_nap'].eq(T), 'y'], label=label)

# Aesthetics
plt.title('Tendencias antes del tratamiento')
plt.xlabel('Días desde el tratamiento')
plt.ylabel('Productividad')
plt.legend()
try:
    plt.savefig('../fig/f36-parallel.png', dpi=200, bbox_inches='tight')
except:
    True
plt.show()

ATT por día

In [ ]:
# Centrar tiempo y crear indicador binario
d['t'] = d['day_in_study'] - 8
d['post'] = np.where(d['day_in_study'] >= 9, 1, 0)

# Crear copia de datos
t = d.copy()

# Crear dummies por periodo
s = pd.get_dummies(t[['t']], columns=['t'], drop_first=False, prefix_sep='')

# Eliminar t0
s.drop(columns=['t0'], inplace=True)

# Interacciones tiempoXtratamiento
s2 = s.multiply(t['T_nap'], axis=0)
s2.columns = [f'{col}*T_nap' for col in s2.columns]

# Pegar a datos
t = pd.concat([t, s, s2], axis=1)

# Asignar constante
t['const'] = 1

# PanelOLS Index
t.set_index(['pid','t'], inplace=True)

# Covariates
X = [
    'const','T_nap','t-7','t-6','t-5','t-4','t-3','t-2','t-1','t1','t2','t3','t4','t5','t6','t7','t8','t9',
    't10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20','t-7*T_nap','t-6*T_nap','t-5*T_nap',
    't-4*T_nap','t-3*T_nap','t-2*T_nap','t-1*T_nap','t1*T_nap','t2*T_nap','t3*T_nap','t4*T_nap','t5*T_nap',
    't6*T_nap','t7*T_nap','t8*T_nap','t9*T_nap','t10*T_nap','t11*T_nap','t12*T_nap','t13*T_nap','t14*T_nap',
    't15*T_nap','t16*T_nap','t17*T_nap','t18*T_nap','t19*T_nap','t20*T_nap'
]

# PanelOLS
m = PanelOLS(
    dependent=t['productivity'],
    exog=t[X]
).fit(cov_type='clustered', cluster_entity=True)

In [ ]:
# Regresión a tabla
res = pd.DataFrame({'tau':m.params.index, 'est':m.params.values, 'se':m.std_errors.values}, index=range(len(m.params)))
res = pd.merge(pd.DataFrame({'tau':[f't{str(i)}*T_nap' for i in range(-7, 21)]}), res, on='tau', how='left')
# res = res.fillna(0)

# Graficar
plt.errorbar(
    x=range(-7,21), y=res['est'], yerr=res['se']*1.96,
    capsize=3, capthick=1, ecolor='C0', marker='o',
    markerfacecolor='C1', markeredgecolor='C1'
)
plt.axvline(0, ls='--', lw=1, color='C0')
plt.axhline(0, ls='-', lw=1, color='black')
plt.xlabel('Días desde tratamiento')
plt.ylabel('Productividad')

# Save
try:
    plt.savefig('../fig/f36-prod.png', dpi=200, bbox_inches='tight')
except:
    True
# Show
plt.show()

ATT con tiempo binario

In [ ]:
# ATT con t binario
t = d.groupby(['T_nap','post'])['productivity'].mean().reset_index(name='prod')
t = t.pivot(index='post', columns='T_nap', values='prod')

# ATT
att = (t.iloc[1,1] - t.iloc[0,1]) - (t.iloc[1,0] - t.iloc[0,0])
print(f'ATT = {round(att, 2)}')

# Print table
# print(t.to_latex(label='t_prod_bin', caption='ATT sobre productividad', float_format='%.2f'))

ATT through time sobre el resto de las variables

In [ ]:
for y in ['nap_time_mins','sleep_report','happy','cog','typing_time_hr']:
    # All effects into one frame
    s1 = pd.get_dummies(d[['t']], columns=['t'], prefix_sep='').drop('t0', axis=1)
    s2 = s1.multiply(d['T_nap'], axis=0)
    s2.columns = [f'{col}*T_nap' for col in s2.columns]
    t = pd.concat([d, s1, s2.assign(const = 1)], axis=1)

    # Reshape
    t = t.set_index(['pid','t'])

    # Fit model
    m = PanelOLS(
        dependent=t[y],
        exog=t[X]
    ).fit(cov_type='clustered', cluster_entity=True)

    # To table
    res = pd.DataFrame({'tau':[f't{str(i)}*T_nap' for i in range(-7, 21)]})
    res = res.merge(
        pd.DataFrame({'tau':m.params.index, 'est':m.params.values, 'se':m.std_errors.values}),
        on='tau', how='left'
    )

    # Plot
    plt.errorbar(
        x=range(-7,21), y=res['est'], yerr=res['se']*1.96,
        capsize=3, capthick=1, ecolor='C0', marker='o',
        markerfacecolor='C1', markeredgecolor='C1'
    )
    plt.axvline(0, ls='--', lw=1, color='C0')
    plt.axhline(0, ls='-', lw=1, color='black')
    plt.xlabel('Días desde tratamiento')
    plt.ylabel(y)

    # Save
    try:
        plt.savefig(f'../fig/f36-{y}.png', dpi=200, bbox_inches='tight')
    except:
        True
    # Show
    plt.show()
    

Efectos con tiempo binario

In [ ]:
l = []
for y in ['nap_time_mins','sleep_report','happy','cog','typing_time_hr']:
    # Datos para PanelOLS
    t = d.assign(
        const = 1,
        t = d['post'],
        T_napXpost = d['T_nap'].multiply(d['post'])
    )

    # t 
    t = t.groupby(['pid','t'])[[y,'const','T_nap','post','T_napXpost']].mean()
    
    # Fit
    m = PanelOLS(
        dependent=t[y],
        exog=t[['const','T_nap','post','T_napXpost']]
    ).fit(cov_type='clustered', cluster_entity=True)

    # Append
    l.append(
        (y, m.params['T_napXpost'], m.std_errors['T_napXpost'], m.pvalues['T_napXpost'])
    )

# Results
res = pd.DataFrame(data=l, columns=['outcome','tau','se','p'])
res

# latex
print(res.to_latex(float_format='%.2f', caption='ATTs con tiempo binario', label='t363', index=False))